In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re
import unicodedata

%precision 3
%matplotlib inline

In [2]:
sns.set()
sns.set_style('darkgrid',{"axes.facecolor": ".9"})
sns.set_context('notebook')

In [3]:
from functions import *

# データの読み込み・整形

In [19]:
'''************************************************************
** データの読み込み
************************************************************'''
df_columns = ('name', 'play', 'like', 'creator', 'size', 'date', 'description')
df_raw = pd.read_csv('data/data_gamejam_2.csv', encoding="utf_8_sig", names=df_columns)
# 区切り行を検出
df = df_raw[df_raw['name'] != "page"]
df.reset_index(inplace=True, drop=True)

'''************************************************************
** 文字列の整形
************************************************************'''
index = 0
raw_p = df['play']
p = []
for d in raw_p:
    i = int(str(d).replace('.', ''))
    tmp = kans2num(str(i))
    p.append(tmp*10)
    index += 1
df['play'] = p


'''************************************************************
** データ型の変換
************************************************************'''
df['like'] = df['like'].astype(int)
df['play'] = df['play'].astype(int)
df['size'] = df['size'].str[:-2]
df['size'] = df['size'].astype(int)
df['date'] = df['date'].str.split('（', expand=True)
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')


'''************************************************************
** 外れ値の除去
************************************************************'''
# labels_outlier = ['play', 'like']

# for l in labels_outlier:
#     s = df[l]
#     bottom, up = findOutlier(s)
#     index = np.where((s <= bottom) | (up <= s))
#     print(bottom, up)
#     print(len(index))
#     df.drop(df.index[index[0]], inplace=True)

df

/Users/tetsuro/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/tetsuro/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/tetsuro/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,name,play,like,creator,size,date,description
0,MirrorRoom,5,0,草餅,0,2021-08-16,This is a room with a mirror. 鏡のある部屋です。
1,ジオメトリック跳梁跋扈,47,2,すずしき,25,2021-08-16,迫りくる敵から魔方陣を守れ！ 目指せ武器コンプリート(全6種) レベルは一気に上げすぎないの...
2,Color Dot Gun!!,31,7,suzuhashi,5,2021-08-16,ドット絵が描けるおもちゃの銃で遊べるワールドです。 32×32マス、31色+透明対応 長押し...
3,JUMP_SQUARE,9,0,hinoki_rocker,35,2021-08-16,#ClusterGAMEJAM 上のフロアを目指しましょう
4,"Hello world, You are just beautiful though cruel.",43,3,わたり,30,2021-08-16,初投稿の作品です。練習用に気になった表現をいろいろ試して作ってます。一人称視点がおすすめです...
5,都市脱出,17,0,水音 凪,4,2021-08-16,アセットとテレポーテーションギミックを使ったタイムアタック脱出ゲーム ギミックは未検証なので...


In [20]:
df.describe()

,play,like,size
count,6.000000,6.00000,6.000000
mean,25.333333,2.00000,16.500000
std,17.682383,2.75681,15.215124
min,5.000000,0.00000,0.000000
25%,11.000000,0.00000,4.250000
50%,24.000000,1.00000,15.000000
75%,40.000000,2.75000,28.750000
max,47.000000,7.00000,35.000000
